## Set up the experiment folder

In [1]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'driver-training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/porto_seguro_safe_driver_prediction_input.csv', os.path.join(training_folder, "porto_seguro_safe_driver_prediction_input.csv"))


'driver-training/porto_seguro_safe_driver_prediction_input.csv'

## train.py
This file defines the key functions required to train the model.  
The file can be invoked with `python train.py` for development purposes.

In [2]:
%%writefile $training_folder/train.py
import os
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import lightgbm
from sklearn.metrics import f1_score
from sklearn.metrics import *


def split_data(data_df):
    """Split a dataframe into training and validation datasets"""
    
    ## TODO
    features = data_df.drop(['target', 'id'], axis = 1)
    labels = np.array(data_df['target'])
    features_train, features_valid, labels_train, labels_valid = train_test_split(features, labels, test_size=0.2, random_state=0)
    
    train_data = lightgbm.Dataset(features_train, label=labels_train)
    valid_data = lightgbm.Dataset(features_valid, label=labels_valid, free_raw_data=False)
    
    return (train_data, valid_data)


def train_model(data, parameters):
    """Train a model with the given datasets and parameters"""
    # The object returned by split_data is a tuple.
    # Access train_data with data[0] and valid_data with data[1]
    
    ## TODO
    model = lightgbm.train(parameters,
                           data[0],
                           valid_sets=data[1],
                           num_boost_round=500,
                           early_stopping_rounds=20)
    
    return model


def get_model_metrics(model, data):
    """Construct a dictionary of metrics for the model"""
    
    ## TODO
    predictions = model.predict(data[1].data)
    fpr, tpr, thresholds = metrics.roc_curve(data[1].label, predictions)
    lgb_prediction = predictions.argmax(axis=0)
    lgb_F1 = f1_score(data[1].label, data[1].label, average='weighted')
    # roc_auc_score(lgb_prediction, data[1].data)
    # recall_score(lgb_prediction, data[1].data)
    model_metrics = {"auc": (metrics.auc(fpr, tpr)), "f1score": lgb_F1}
    print(model_metrics)
    
    return model_metrics


def main():
    """This method invokes the training functions for development purposes"""
    
    # Read data from a file
    data_df = pd.read_csv('porto_seguro_safe_driver_prediction_input.csv')

    # Hard code the parameters for training the model
    parameters = {
        'learning_rate': 0.02,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'sub_feature': 0.7,
        'num_leaves': 60,
        'min_data': 100,
        'min_hessian': 1,
        'verbose': 2
    }

    # Call the functions defined in this file
    data = split_data(data_df)
    model = train_model(data, parameters)
    predictions = get_model_metrics(model,data)
    
    # Print the resulting metrics for the model
    ## TODO
    print(predictions)
    
if __name__ == '__main__':
    main()


Writing driver-training/train.py


## parameters.json
This file will specify the parameters used to train the model.

In [3]:
%%writefile $training_folder/parameters.json
{
    "training":
    {
        "learning_rate": 0.02,
        "boosting_type": "gbdt",
        "objective": "binary",
        "metric": "auc",
        "sub_feature": 0.7,
        "num_leaves": 60,
        "min_data": 100,
        "min_hessian": 1,
        "verbose": 0
    }
}


Writing driver-training/parameters.json


## driver_training.py
This file will be the entry script when running an Azure ML context.  
It calls the functions defined in train.py for data preparation and training, but reads parameters from a file, and logs output to the Azure ML context.  
The file can be invoked with `python driver_training.py` for development purposes.

In [4]:
%%writefile $training_folder/driver_training.py
# Import libraries
import argparse
from azureml.core import Run
import joblib
import json
import os
import pandas as pd
import shutil

# Import functions from train.py
from train import split_data, train_model, get_model_metrics

# Get the output folder for the model from the '--output_folder' parameter
parser = argparse.ArgumentParser()
parser.add_argument('--output_folder', type=str, dest='output_folder', default="outputs")
args = parser.parse_args()
output_folder = args.output_folder

# Get the experiment run context
run = Run.get_context()

# load the safe driver prediction dataset
train_df = pd.read_csv('porto_seguro_safe_driver_prediction_input.csv')

# Load the parameters for training the model from the file
with open("parameters.json") as f:
    pars = json.load(f)
    parameters = pars["training"]

# Log each of the parameters to the run
for param_name, param_value in parameters.items():
    run.log(param_name, param_value)
    
# Use the functions imported from train.py to prepare data, train the model, and calculate the metrics
## TODO
data = split_data(train_df)

model = train_model(data, parameters)
predictions = get_model_metrics(model,data)

run.log('Accuracy', predictions)
run.log('auc', predictions['auc'])
run.log('f1score', predictions['f1score'])

# Save the trained model to the output folder
os.makedirs(output_folder, exist_ok=True)
output_path = output_folder + "/driver_model.pkl"
joblib.dump(value=model, filename=output_path)

run.complete()

Writing driver-training/driver_training.py


In [5]:
import azureml.core
from azureml.core import Workspace

# Load the workspace
ws = Workspace.from_config()

## Use an Estimator to Run the Script as an Experiment

See [this tutorial](https://github.com/MicrosoftDocs/mslearn-aml-labs/blob/master/02-Training_Models.ipynb) for a starting point

Use the scikit-learn and lightgbm conda packages

In [6]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment

# Create an estimator
estimator = Estimator(source_directory=training_folder,
                      entry_script='driver_training.py',
                      compute_target='local',
                      conda_packages=['scikit-learn','lightgbm']
                      )

# Create an experiment
experiment_name = 'driver-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment based on the estimator
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

'Estimator' is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or an Azure ML curated environment.


RunId: driver-training_1611507501_da264895
Web View: https://ml.azure.com/experiments/driver-training/runs/driver-training_1611507501_da264895?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/workspaces/mlopsdev

Streaming azureml-logs/60_control_log.txt

[2021-01-24T16:58:25.844629] Entering context manager injector.
[2021-01-24T16:58:26.481061] Using urllib.request Python 3.0 or later
Streaming log file azureml-logs/60_control_log.txt
Starting the daemon thread to refresh tokens in background for process with pid = 7501
Running: ['/bin/bash', '/tmp/azureml_runs/driver-training_1611507501_da264895/azureml-environment-setup/docker_env_checker.sh']

Materialized image not found on target: azureml/azureml_ca5cde6fc63b668f23d793c46084bb15


[2021-01-24T16:58:30.206218] Logging experiment preparation status in history service.
Running: ['/bin/bash', '/tmp/azureml_runs/driver-training_1611507501_da264895/azureml-environment-setup/docker_env_builder.sh']
Running:

{'runId': 'driver-training_1611507501_da264895',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-01-24T17:01:52.129907Z',
 'endTimeUtc': '2021-01-24T17:02:16.107415Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '69f98fb6-05d7-4ae6-b856-dcb54b1701fd',
  'azureml.git.repository_uri': 'https://csedevops@dev.azure.com/csedevops/DSDevOpsOH/_git/DSDevOpsOH',
  'mlflow.source.git.repoURL': 'https://csedevops@dev.azure.com/csedevops/DSDevOpsOH/_git/DSDevOpsOH',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'a378d3666581824cd0c4358d463a4c98af630878',
  'mlflow.source.git.commit': 'a378d3666581824cd0c4358d463a4c98af630878',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'driver_training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': '

In [7]:
# Print the resulting metrics
metrics = run.get_metrics()
for k, v in metrics.items():
        print(k, v)

learning_rate 0.02
boosting_type gbdt
objective binary
metric auc
sub_feature 0.7
num_leaves 60
min_data 100
min_hessian 1
verbose 0
Accuracy {'auc': 0.6377511613946426, 'f1score': 1.0}
auc 0.6377511613946426
f1score 1.0


In [8]:
# Register the model
run.register_model(model_path='outputs/driver_model.pkl', model_name='driver_model.pkl',
                  tags={'Accuracy': run.get_metrics()['Accuracy'], 'learningrate' : run.get_metrics()['learning_rate'], 'f1score' : run.get_metrics()['f1score'], 'auc' : run.get_metrics()['auc']})

Model(workspace=Workspace.create(name='mlopsdev', subscription_id='c46a9435-c957-4e6c-a0f4-b9a597984773', resource_group='mlops'), name=driver_model.pkl, id=driver_model.pkl:8, version=8, tags={'Accuracy': "{'auc': 0.6377511613946426, 'f1score': 1.0}", 'learningrate': '0.02', 'f1score': '1.0', 'auc': '0.6377511613946426'}, properties={})

In [9]:
# Register the model
run.register_model(model_path='outputs/driver_model.pkl', model_name='driver_model.pkl')

Model(workspace=Workspace.create(name='mlopsdev', subscription_id='c46a9435-c957-4e6c-a0f4-b9a597984773', resource_group='mlops'), name=driver_model.pkl, id=driver_model.pkl:9, version=9, tags={}, properties={})